In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def transform_county(dfname, columnname):
    
    dfname[columnname] = dfname[columnname].str.lower().replace(to_replace=r' county',value='',regex=True)

def transform_number(dfname,columnname):
    dfname[columnname] = dfname[columnname].replace(to_replace=r',',value='',regex=True)

In [3]:
pop_2010 = pd.read_csv("population-econdata/2010_County_Population.csv", encoding = "ISO-8859-1")
pop_2010 = pop_2010[pop_2010['State'].str.lower()!=pop_2010['County'].str.lower()]
transform_county(pop_2010, "County")


pop_2010.head()

,State,County,2010_Census_Population
1,Alabama,autauga,54571
2,Alabama,baldwin,182265
3,Alabama,barbour,27457
4,Alabama,bibb,22915
5,Alabama,blount,57322


In [4]:
county_gdp = pd.read_csv("population-econdata/County_GDP.csv")


transform_county(county_gdp, "County")

county_gdp.head()

,STCOUNTYFP,County,State,2004_GDP,2005_GDP
0,1001,autauga,AL,1170278,1195319
1,1003,baldwin,AL,4912303,5502688
2,1005,barbour,AL,888479,892271
3,1007,bibb,AL,324760,322815
4,1009,blount,AL,863041,887578


In [5]:
zip_county = pd.read_csv("population-econdata/ZIP_County_FIPS.csv")

transform_county(zip_county, "COUNTYNAME")
  
zip_county.head()

,STATE,ZIP,COUNTYNAME,Unnamed: 3,STCOUNTYFP,CLASSFP
0,AL,36003,autauga,NaN,1001,H1
1,AL,36006,autauga,NaN,1001,H1
2,AL,36067,autauga,NaN,1001,H1
3,AL,36066,autauga,NaN,1001,H1
4,AL,36703,autauga,NaN,1001,H1


In [6]:
county_pov_income = pd.read_csv("population-econdata/County_Poverty_Income.csv")
county_pov_income = county_pov_income[county_pov_income['County_Name'].str.contains('County')]
transform_county(county_pov_income, "County_Name")
transform_number(county_pov_income, "2005_Poverty_All_Ages")
transform_number(county_pov_income, "2005_Median_HH_Income")


county_pov_income.head()

,State,County_Name,2005_Poverty_All_Ages,2005_Poverty_Percent_All_Ages,2005_Poverty_Under18,2005_Poverty_Percent_Under18,2005_Median_HH_Income
1,AL,autauga,5020,10.4,"1,828",14.7,45019
2,AL,baldwin,18249,11.4,"6,778",18.9,42804
3,AL,barbour,5694,22.4,"2,035",31.5,29534
4,AL,bibb,3364,16.6,"1,223",24.2,34212
5,AL,blount,6248,11.4,"2,160",16.7,40588


In [7]:
zip_county.ZIP.nunique()

39456

In [8]:
print(zip_county.shape[0])

print(county_gdp.shape[0])

52889
3118


In [9]:
df_merged = pd.merge(zip_county, county_gdp, on="STCOUNTYFP",how='inner') 
print(len(df_merged))
df_merged.head()

52048


,STATE,ZIP,COUNTYNAME,Unnamed: 3,STCOUNTYFP,CLASSFP,County,State,2004_GDP,2005_GDP
0,AL,36003,autauga,NaN,1001,H1,autauga,AL,1170278,1195319
1,AL,36006,autauga,NaN,1001,H1,autauga,AL,1170278,1195319
2,AL,36067,autauga,NaN,1001,H1,autauga,AL,1170278,1195319
3,AL,36066,autauga,NaN,1001,H1,autauga,AL,1170278,1195319
4,AL,36703,autauga,NaN,1001,H1,autauga,AL,1170278,1195319


In [10]:
len(df_merged.groupby(['STATE','County']))

3087

In [11]:
states=pd.read_csv('population-econdata/states.csv',header=None, names=['FULL','STATE','1','2'])
states.head()



,FULL,STATE,1,2
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [12]:
df_merged=pd.merge(df_merged, states, on="STATE",how='left')
df_merged.head()

,STATE,ZIP,COUNTYNAME,Unnamed: 3,STCOUNTYFP,CLASSFP,County,State,2004_GDP,2005_GDP,FULL,1,2
0,AL,36003,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central
1,AL,36006,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central
2,AL,36067,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central
3,AL,36066,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central
4,AL,36703,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central


In [13]:
#pop_2010.head()
print(len(pop_2010))
pop_2010.groupby(['State','County']).agg(count=('2010_Census_Population','count')).reset_index().sort_values(by='count',ascending=False)

3141


,State,County,count
0,Alabama,autauga,1
2087,Ohio,logan,1
2089,Ohio,lucas,1
2090,Ohio,madison,1
2091,Ohio,mahoning,1
...,...,...,...
1049,Kentucky,johnson,1
1050,Kentucky,kenton,1
1051,Kentucky,knott,1
1052,Kentucky,knox,1


In [14]:
## !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

print(len(df_merged))
df_merged = pd.merge(df_merged, pop_2010, left_on = ["County","FULL"], right_on=["County","State"], how="inner")
print(len(df_merged))
df_merged.head()

52048
50624


,STATE,ZIP,COUNTYNAME,Unnamed: 3,STCOUNTYFP,CLASSFP,County,State_x,2004_GDP,2005_GDP,FULL,1,2,State_y,2010_Census_Population
0,AL,36003,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central,Alabama,54571
1,AL,36006,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central,Alabama,54571
2,AL,36067,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central,Alabama,54571
3,AL,36066,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central,Alabama,54571
4,AL,36703,autauga,NaN,1001,H1,autauga,AL,1170278,1195319,Alabama,South,East South Central,Alabama,54571


In [15]:
#df_merged=df_merged.drop_duplicates(subset=['STATE','ZIP','County'])
#len(df_merged)

In [16]:
df_merged.drop(columns=["COUNTYNAME", "State_x", "State_y", "CLASSFP","1","2",'Unnamed: 3'], inplace=True)

df_merged.head()

,STATE,ZIP,STCOUNTYFP,County,2004_GDP,2005_GDP,FULL,2010_Census_Population
0,AL,36003,1001,autauga,1170278,1195319,Alabama,54571
1,AL,36006,1001,autauga,1170278,1195319,Alabama,54571
2,AL,36067,1001,autauga,1170278,1195319,Alabama,54571
3,AL,36066,1001,autauga,1170278,1195319,Alabama,54571
4,AL,36703,1001,autauga,1170278,1195319,Alabama,54571


In [17]:
print(len(df_merged))
df_merged = pd.merge(df_merged,county_pov_income, left_on = ['STATE',"County"], right_on = ['State',"County_Name"],how='inner')
print(len(df_merged))
df_merged.head()

50624
50290


,STATE,ZIP,STCOUNTYFP,County,2004_GDP,2005_GDP,FULL,2010_Census_Population,State,County_Name,2005_Poverty_All_Ages,2005_Poverty_Percent_All_Ages,2005_Poverty_Under18,2005_Poverty_Percent_Under18,2005_Median_HH_Income
0,AL,36003,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019
1,AL,36006,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019
2,AL,36067,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019
3,AL,36066,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019
4,AL,36703,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019


In [18]:
df_merged=df_merged.drop_duplicates(subset=['STATE','ZIP','County'])
df_merged.head()


,STATE,ZIP,STCOUNTYFP,County,2004_GDP,2005_GDP,FULL,2010_Census_Population,State,County_Name,2005_Poverty_All_Ages,2005_Poverty_Percent_All_Ages,2005_Poverty_Under18,2005_Poverty_Percent_Under18,2005_Median_HH_Income
0,AL,36003,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019
1,AL,36006,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019
2,AL,36067,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019
3,AL,36066,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019
4,AL,36703,1001,autauga,1170278,1195319,Alabama,54571,AL,autauga,5020,10.4,"1,828",14.7,45019


In [19]:
df_merged.drop(columns=["State", "2005_Poverty_Percent_All_Ages", "2005_Poverty_Under18", 
                        "2005_Poverty_Percent_Under18",
                        "County_Name"], 
              inplace = True)


df_merged.head()

,STATE,ZIP,STCOUNTYFP,County,2004_GDP,2005_GDP,FULL,2010_Census_Population,2005_Poverty_All_Ages,2005_Median_HH_Income
0,AL,36003,1001,autauga,1170278,1195319,Alabama,54571,5020,45019
1,AL,36006,1001,autauga,1170278,1195319,Alabama,54571,5020,45019
2,AL,36067,1001,autauga,1170278,1195319,Alabama,54571,5020,45019
3,AL,36066,1001,autauga,1170278,1195319,Alabama,54571,5020,45019
4,AL,36703,1001,autauga,1170278,1195319,Alabama,54571,5020,45019


In [20]:
df_merged.dtypes

STATE                     object
ZIP                        int64
STCOUNTYFP                 int64
County                    object
2004_GDP                  object
2005_GDP                  object
FULL                      object
2010_Census_Population     int64
2005_Poverty_All_Ages     object
2005_Median_HH_Income     object
dtype: object

In [21]:
print(len(df_merged))

50290


In [22]:
df_merged['2004_GDP']=pd.to_numeric(df_merged['2004_GDP'],'coerce')
df_merged=df_merged.dropna()
print(len(df_merged))
df_merged['2005_GDP']=pd.to_numeric(df_merged['2005_GDP'],'coerce')
df_merged=df_merged.dropna()
print(len(df_merged))
df_merged['2005_Poverty_All_Ages']=pd.to_numeric(df_merged['2005_Poverty_All_Ages'],'coerce')
df_merged=df_merged.dropna()
print(len(df_merged))
df_merged['2005_Median_HH_Income']=pd.to_numeric(df_merged['2005_Median_HH_Income'],'coerce')
df_merged=df_merged.dropna()
print(len(df_merged))

50290
50290
50290
50290


In [23]:
df_merged=df_merged.groupby('ZIP').agg(pop_sum=('2010_Census_Population','sum'),
                                       GDP2004_sum=('2004_GDP','sum'),
                                       GDP2005_sum=('2005_GDP','sum'),
                                       poverty_sum=('2005_Poverty_All_Ages','sum'),
                                        median_income=('2005_Median_HH_Income','mean')
                                       ).reset_index()
print(len(df_merged))
print(df_merged.ZIP.nunique())
df_merged

37575
37575


,ZIP,pop_sum,GDP2004_sum,GDP2005_sum,poverty_sum,median_income
0,501,1493350,74117244,74545223,74833,76697.0
1,1001,463490,18994298,19035742,73233,41998.0
2,1002,229452,8349269,8423852,23283,47989.0
3,1003,158080,5719878,5807855,15394,49233.0
4,1004,158080,5719878,5807855,15394,49233.0
...,...,...,...,...,...,...
37570,99363,58781,2275286,2349144,9716,37726.0
37571,99371,96891,2653740,3030911,14002,38931.5
37572,99401,21623,493096,522287,3652,36096.0
37573,99402,21623,493096,522287,3652,36096.0


In [ ]:
df_merged.to_csv("econ-popdata.csv")